In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda:0"

In [ ]:
# Load SDXL model
from diffusers import DiffusionPipeline
import torch

base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda")

In [ ]:
import os
from tqdm import tqdm
out_path = "../data/portraits_sdxl_ethnicity/"  # "../data/portraits_sdxl_gender/"
# prompt = "A headshot of a man|woman in his 40s, DSLR, detailed, 8k, in perfect focus, shoulder height"
prompt = "A headshot of a black person|caucasian person, DSLR, detailed, 8k, in perfect focus, shoulder height"

for i in tqdm(range(2)):
    image = base(
        prompt=prompt,
        num_inference_steps=50,
        denoising_end=0.8,
        output_type="latent",
    ).images
    image = refiner(
        prompt=prompt,
        num_inference_steps=50,
        denoising_start=0.8,
        image=image,
    ).images[0]
    image.save(os.path.join(out_path, f"portrait_{50+i}.jpg"))